In [ ]:
from google.colab import drive
import pandas as pd
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
pip install geopandas shapely

In [ ]:
pip install pandas requests

In [ ]:
import geopandas as gpd
from shapely.geometry import Point

# Cargar el archivo GeoJSON
gdf = gpd.read_file("/content/drive/MyDrive/Codigo Tesis Helicobacter Editado/Datos/Veredas_de_Colombia.geojson")

# Convertir grados, minutos y segundos a decimal como se explicó anteriormente
def convertir_a_decimal(grados, minutos, segundos):
    return grados + (minutos / 60) + (segundos / 3600)

In [ ]:
url=("/content/drive/MyDrive/Codigo Tesis Helicobacter Editado/Datos/info_vereda_1.xlsx")
df1 = pd.read_excel(io=url,sheet_name="Vereda")

In [ ]:
url='/content/drive/MyDrive/Codigo Tesis Helicobacter Editado/Datos/BD_Editada.xlsx'
dfH=pd.read_excel(url,sheet_name='HOGAR', comment='#')
dfH.shape
dfH = pd.DataFrame(dfH)

In [ ]:
# Convertir a coordenadas decimales
dfH['Longitud'] = dfH.apply(lambda row: -convertir_a_decimal(row['Longitud (W), Grados:'], row['Longitud (W), Minutos:'], row['Longitud (W), Segundos:']), axis=1)
dfH['Latitud'] = dfH.apply(lambda row: convertir_a_decimal(row['Latitud (N), Grados:'], row['Latitud (N), Minutos:'], row['Latitud (N), Segundos:']), axis=1)

# Crear puntos geográficos
dfH['Coordinates'] = dfH.apply(lambda row: Point(row['Longitud'], row['Latitud']), axis=1)

# Convertir DataFrame a GeoDataFrame
geo_df = gpd.GeoDataFrame(dfH, geometry='Coordinates')

# Hacer la consulta espacial
point_in_poly = gpd.sjoin(geo_df, gdf, how="left", op='intersects')

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3473: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if (await self.run_code(code, result,  async_=asy)):
<ipython-input-7-89653649c39c>:12: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: EPSG:4326

  point_in_poly = gpd.sjoin(geo_df, gdf, how="left", op='intersects')


In [ ]:
url='/content/drive/MyDrive/Codigo Tesis Helicobacter Editado/Datos/BD_Editada.xlsx'
dfE=pd.read_excel(url,sheet_name='Extras', comment='#')
dfE.shape
dfE = pd.DataFrame(dfE)

In [ ]:
# Crear puntos geográficos
dfE['Coordinates'] = dfE.apply(lambda row: Point(row['Longitud'], row['Latitud']), axis=1)

# Convertir DataFrame a GeoDataFrame
geo_df = gpd.GeoDataFrame(dfE, geometry='Coordinates')

# Hacer la consulta espacial
point_in_polyE = gpd.sjoin(geo_df, gdf, how="left", op='intersects')

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3473: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if (await self.run_code(code, result,  async_=asy)):
<ipython-input-9-66e6b441b01f>:8: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: EPSG:4326

  point_in_polyE = gpd.sjoin(geo_df, gdf, how="left", op='intersects')


In [ ]:
point_in_polyE = point_in_polyE[point_in_poly.columns]

In [ ]:
# Concatenar DataFrames
df_concatenado = pd.concat([point_in_poly, point_in_polyE], ignore_index=True)

In [ ]:
df1.rename(columns={'CÓDIGO VEREDA': 'CODIGO_VER'}, inplace=True)

In [ ]:
df_concatenado['CODIGO_VER']=pd.to_numeric(df_concatenado['CODIGO_VER'])

In [ ]:
df_concatenado=pd.merge(df_concatenado,df1, on='CODIGO_VER',how='left')

In [ ]:
# Columnas a seleccionar además de COD_HOGAR
columnas_a_seleccionar = ['UP', 'UPA', 'UPNA', 'VIVIENDAS', 'HOGARES', 'PERSONAS', 'PRODUCTORES RESIDENTES', 'CodHogar']

# Crear un nuevo DataFrame con las columnas seleccionadas
nuevo_df = df_concatenado[columnas_a_seleccionar].copy()
df_concatenado=df_concatenado.drop(['UP', 'UPA', 'UPNA', 'VIVIENDAS', 'HOGARES', 'PERSONAS', 'PRODUCTORES RESIDENTES'], axis=1)

In [ ]:
# Unimos y sumamos
resultado = nuevo_df.groupby('CodHogar').sum()

In [ ]:
df_concatenado = df_concatenado.drop_duplicates(subset='CodHogar', keep='first')

In [ ]:
df_concatenado=pd.merge(df_concatenado,resultado, on='CodHogar')

In [ ]:
 df_concatenado=df_concatenado.drop(['Latitud (N), Grados:','Latitud (N), Minutos:', 'Latitud (N), Segundos:','Longitud (W), Grados:', 'Longitud (W), Minutos:','Longitud (W), Segundos:', 'Coordinates', 'index_right',], axis=1 )

In [ ]:
# Filtrar el DataFrame para excluir filas donde la columna 'PERSONAS' es igual a 0
df_concatenado = df_concatenado[df_concatenado['PERSONAS'] != 0]

In [ ]:
# Guardar en nuevo archivo CSV
df_concatenado.to_csv('/content/drive/MyDrive/Codigo Tesis Helicobacter Editado/Datos/data_concatenada.csv', index=False)